In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
import pyspark
import pandas as pd
import os
import requests
from datetime import datetime
from urllib.request import urlopen
from trino.dbapi import connect
from sqlalchemy import create_engine
from sqlalchemy.schema import Table, MetaData
from sqlalchemy.sql.expression import select, text

In [4]:
engine = create_engine("trino://hive:@192.168.10.210:8090/hive/transaction")
connection = engine.connect()

In [5]:
os.environ['HADOOP_CONF_DIR'] = '/etc/hadoop/conf'
os.environ['JAVA_HOME'] = '/usr/local/jdk8u222-b10'
os.environ['HADOOP_USER_NAME']='hive'
os.environ['PYSPARK_PYTHON'] ='/HDFS01/anaconda3/envs/main/bin/python'
conf = pyspark.SparkConf().setAll([
     ('spark.driver.maxResultSize', '0'),
     ('spark.driver.memory', '2g'),
     ('spark.sql.repl.eagerEval.enabled','true'),
     ('hive.strict.managed.tables','false'),
     ('hive.metastore.uris', 'thrift://nn01.bigdata:9083'),
     ('metastore.client.capability.check','false')
    ])
spark = SparkSession.builder \
        .master("local[*]") \
        .appName("testreplace") \
        .config(conf=conf) \
        .enableHiveSupport() \
        .getOrCreate();

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/10 09:12:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/03/10 09:12:57 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/03/10 09:12:57 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
23/03/10 09:12:57 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
23/03/10 09:12:57 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
23/03/10 09:12:57 WARN Utils: Service 'SparkUI' could not bind on port 4044. Attempting port 4045.
23/03/10 09:12:57 WARN Utils: Service 'SparkUI' could not bind on port 4045. Attempting port 4046.
23/03/10 09:12:57 WARN Utils: Service 'SparkUI' could not bind on port 4046. Attempting port 4047.


In [ ]:
schema = StructType([
    StructField("job_id", StringType(), True),
    StructField("addjob_by", StringType(), True),
    StructField("customer", StringType(), True),
    StructField("site", StringType(), True),
    StructField("malltype", StringType(), True),
    StructField("worktype", StringType(), True),
    StructField("details", StringType(), True),
    StructField("onsite", StringType(), True),
    StructField("time", StringType(), True),
    StructField("travel", StringType(), True),
    StructField("engineer", StringType(), True),
    StructField("status", StringType(), True),
    StructField("day", StringType(), True),
    StructField("date_start", StringType(), True),
    StructField("date_end", StringType(), True),
    StructField("status_work", StringType(), True),
    StructField("end_job_detail", StringType(), True),
    StructField("file_location", StringType(), True),
    StructField("update_by", StringType(), True)
])

In [ ]:
df = spark.createDataFrame(pd.read_json('http://engineer.da.co.th/api_json.php'), schema=schema)
df = df.withColumn("details", regexp_replace("details", "\r\n", ""))

In [ ]:
df.write \
.mode('append') \
.option("sep","|") \
.csv(f"/user/hive/warehouse/pyspark.db/engineer_csv/{today}/{times}")

In [ ]:
os.system("sudo -u hdfs hdfs dfs -rmr /user/hive/warehouse/pyspark.db/engineer_csv")

In [ ]:
os.system("sudo -u hdfs hdfs dfs -ls /user/hive/warehouse/pyspark.db/engineer_csv/20230216/10021676517926")

In [7]:
pd.read_sql(' select * from transaction.covid', con=engine)

,year,weeknum,province,new_case,total_case,new_case_excludeabroad,total_case_excludeabroad,new_death,total_death,update_date
0,2023,9,กระบี่,0,2,0,2,0,0,2023-03-06 07:39:55
1,2023,9,สตูล,0,1,0,1,0,0,2023-03-06 07:39:55
2,2023,9,ลำปาง,1,25,1,25,1,9,2023-03-06 07:39:55
3,2023,9,ตราด,0,23,0,23,0,3,2023-03-06 07:39:55
4,2023,9,ทั้งประเทศ,147,4263,147,4263,7,249,2023-03-06 07:39:55
...,...,...,...,...,...,...,...,...,...,...
74,2023,9,มหาสารคาม,1,28,1,28,0,0,2023-03-06 07:39:55
75,2023,9,สมุทรสงคราม,0,23,0,23,0,0,2023-03-06 07:39:55
76,2023,9,สงขลา,1,39,1,39,0,0,2023-03-06 07:39:55
77,2023,9,ภูเก็ต,0,4,0,4,0,1,2023-03-06 07:39:55
